In [1]:
from src import SQL_Executor, SQL_Queries, Vizualizator
import pandas as pd
import numpy as np

executor = SQL_Executor()
query = SQL_Queries.Payment()
df = executor.execute_query(query.define_dpd_aggregate())
t_df = executor.execute_query(query.calculate_dpd90_trend())

DatabaseError: Execution failed on sql '
            SELECT 
                transaction_id,
                installment_number,
                payment_date,
                scheduled_date
            FROM installments
            ': no such table: installments

In [ ]:
dpd = df.copy()
scheduled_date = pd.to_datetime(dpd["scheduled_date"])
payment_date = pd.to_datetime(dpd["payment_date"])
today = pd.Timestamp.today()

conditions = [
    payment_date > scheduled_date,
    payment_date.isna(),
    payment_date <= scheduled_date,
]

choices = [
    (payment_date - scheduled_date).dt.days,
    np.maximum((today - scheduled_date).dt.days, 0),
    0,
]

dpd["dpd"] = np.select(conditions, choices, default=0)
dpd = dpd.drop(columns=["payment_date", "scheduled_date"])

dpd90 = dpd.copy()
dpd90["dpd90"] = (dpd90["dpd"] >= 90).astype(int)
dpd90 = dpd90.groupby("transaction_id").agg({"dpd90": "max"})
dpd90 = t_df.merge(dpd90, on="transaction_id", how="left")

In [ ]:
bins = [18, 26, 36, 51]
dpd90_rates_age = Vizualizator(dpd90)
dpd90_rates_age.group_by_column(bins, "dpd90", "age", "income")
dpd90_rates_age.visualize(
    column="dpd90_rate",
    kind="bar",
    ylim=(0, 100),
    title="DPD90 by age group",
    rotation=0,
    xlabel="Age group",
    ylabel="%",
);

In [ ]:
bins = [0, 15000, 30000, 60000]
dpd90_rates_income = Vizualizator(dpd90)
dpd90_rates_income.group_by_column(bins, "dpd90", "income", "age")
dpd90_rates_income.visualize(
    column="dpd90_rate",
    kind="bar",
    ylim=(0, 100),
    title="DPD90 by income group",
    rotation=0,
    xlabel="Income group",
    ylabel="%",
)